In [ ]:
#TASK-4(Natural Language Processing for Text classification)

In [11]:
!pip install tensorflow numpy

In [14]:
#Data Preparation
#Importing libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Embedding


In [21]:
# Text data to be used for training the model
text = """Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: 
once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 
'and what is the use of a book,' thought Alice 'without pictures or conversations?' So she was considering in her own mind 
(as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain 
would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.
"""


In [22]:
# Initialize a Tokenizer to convert words to numeric tokens
tokenizer = Tokenizer() 
tokenizer.fit_on_texts([text])  # Fit the tokenizer on the text

# Get the total number of unique words, plus one for padding purposes
total_words = len(tokenizer.word_index) + 1

# Initialize an empty list to store input sequences
input_sequences = []

In [23]:
# Split text into sentences based on periods and create sequences of tokens
for line in text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]  # Convert each sentence to a sequence of tokens
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]  # Create sequences of increasing length
        input_sequences.append(n_gram_sequence)  # Add to list of input sequences


In [26]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Find the maximum sequence length to use for padding
max_sequence_len = max([len(seq) for seq in input_sequences])

# Pad the sequences so they all have the same length
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


In [27]:
# Split data into input (X) and output (y)
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)  # Convert output to categorical


In [29]:
from tensorflow.keras.layers import Dropout, LSTM, Embedding, Dense

# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))  # Embedding layer
model.add(LSTM(150))  # LSTM layer with 150 units
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(total_words, activation='softmax'))  # Output layer with softmax activation


In [30]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model for 50 epochs
epochs = 50
history = model.fit(X, y, epochs=epochs, verbose=1)

Epoch 1/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 156ms/step - accuracy: 0.0157 - loss: 4.3715
Epoch 2/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step - accuracy: 0.0770 - loss: 4.3581
Epoch 3/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step - accuracy: 0.0960 - loss: 4.3422
Epoch 4/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 138ms/step - accuracy: 0.0478 - loss: 4.3092
Epoch 5/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 240ms/step - accuracy: 0.0444 - loss: 4.2251
Epoch 6/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 223ms/step - accuracy: 0.0829 - loss: 4.1168
Epoch 7/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.0579 - loss: 4.1120
Epoch 8/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 140ms/step - accuracy: 0.0346 - loss: 4.0445
Epoch 9/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 230ms/step - accuracy: 0.0617 - loss: 3.9427
Epoch 10/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - accuracy: 0.0802 - loss: 3.9063
Epoch 11/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - accuracy: 0.0736 - loss: 3.8647
Epoch 12/50
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step - accuracy: 0.0857 - lo

In [33]:
# Define a function to generate new text based on a seed text and desired length
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]  # Convert seed text to tokens
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')  # Pad sequence
        predicted = model.predict(token_list, verbose=0)  # Predict the next word
        predicted_word_index = np.argmax(predicted, axis=1)[0]  # Get index of predicted word
        
        # Find the word corresponding to the predicted index
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                seed_text += " " + word  # Append word to seed text
                break
    return seed_text

In [32]:
# Generate and print text starting with a seed text
seed_text = "There was a boy named kevin"
generated_text = generate_text(seed_text, 10, max_sequence_len)
print(f"Generated text: {generated_text}")

# Save the trained model and tokenizer
model.save('text_generation_lstm_model.h5')
import joblib
joblib.dump(tokenizer, 'tokenizer.pkl')


Generated text: There was a boy named kevin was was get very tired of sitting by her sister


['tokenizer.pkl']